In [61]:
from pathlib import Path
import openpyxl as px
import csv
import os
import numpy as np
import pandas as pd

from g00x_figures.data import Data

data = Data()

In [ ]:
# import os

# DATA_DIR = Path("/Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data")

# # Get all .csv and .feather files in the data directory except data.py itself
# all_files = [f for f in DATA_DIR.glob("**/*.csv")] + [f for f in DATA_DIR.glob("**/*.feather")]

# # Get used files from data.paths (second element of each tuple)
# used_files = set(os.path.basename(f) for f in [d[1] for d in data.paths if isinstance(d, (list, tuple)) and len(d) > 1])

# unused_files = [f for f in all_files if f.name not in used_files]

# if unused_files:
#     print("Unused .csv or .feather files in src/g002_figures/data:")
#     for f in unused_files:
#         print(f"- {f}")
#         print(f"Deleting: {f}")
#         f.unlink()  # This permanently deletes the file
# else:
#     print("All .csv and .feather files in src/g002_figures/data are referenced in data.paths")

Unused .csv or .feather files in src/g002_figures/data:
- /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/asc_num_seq.csv
Deleting: /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/asc_num_seq.csv
- /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/flow_and_sequencing_long_names.csv
Deleting: /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/flow_and_sequencing_long_names.csv
- /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/flow.csv
Deleting: /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/flow.csv
- /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/final_df.csv
Deleting: /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/final_df.csv
- /Users/tmsincomb/Dropbox/repos/g002_figures_clean/src/g002_figures/data/g003/seq.csv
Deleting: /Users/tmsincomb/Dropbox/repos/g002_f

In [64]:
files = sorted(list(Path("Main-Metrics").glob("**/*.csv")))
files += list(Path("Table-Metrics").glob("**/*.xlsx"))
files += list(Path("Table-Sup-Metrics").glob("**/*.xlsx"))
files += sorted(list(Path("Sup-Metrics").glob("**/*.csv")))
files[40:50]

[PosixPath('Main-Metrics/fig7/figE_percent_of_VRC01-class_IgG$^{+}$_sequences_with_key_HCDR2_residues_>_1.csv'),
 PosixPath('Main-Metrics/fig7/figF_percent_VRC01-class_responders_with_key_HCDR2_residues_>_1.csv'),
 PosixPath('Main-Metrics/fig7/figG_percent_of_IgG$^{+}$_memory_B_cells_detected_as_VRC01-class_with_key_HCDR2_residues_>_1.csv'),
 PosixPath('Main-Metrics/fig7/figH_Fraction_Unique_Clones_for_core$^{++}$KO$^{-}$_VRC01-class_IgG$^{+}$_BCRs_with_key_HCDR2_residues_>_1.csv'),
 PosixPath('Main-Metrics/fig8/fig8A.csv'),
 PosixPath('Table-Metrics/Table1.xlsx'),
 PosixPath('Table-Sup-Metrics/tableS65.xlsx'),
 PosixPath('Sup-Metrics/S10/2025-02-12G00X_manuscript_Fig10.csv'),
 PosixPath('Sup-Metrics/S19/figA_percent_GT8++_among_IgG+_B_cells.csv'),
 PosixPath('Sup-Metrics/S19/figB_percent_CD4bs-specific_among_IgG+_B_cells.csv')]

In [5]:
def dataframes_to_csv_strings(dfs_list):
    """
    Convert a list of pandas DataFrames into a list of CSV strings

    Args:
        dfs_list: List of pandas DataFrames

    Returns:
        list: List of strings where each string is a CSV representation of a DataFrame
    """
    csv_tab = []
    for df in dfs_list:
        csv_list = [df.columns.tolist()] + df.values.tolist() + [""]
        csv_tab += csv_list

    return csv_tab

In [6]:
from openpyxl import load_workbook
from copy import copy


def copy_sheet_colors(source_path, target_path, source_sheet_name, target_sheet_name):
    """Copies cell colors from a sheet in one workbook to another."""

    # Load workbooks
    source_wb = load_workbook(source_path)
    target_wb = load_workbook(target_path)

    # Get source and target sheets
    source_sheet = source_wb[source_sheet_name]
    target_sheet = target_wb[target_sheet_name]

    # Copy cell colors
    for row_index, row in enumerate(source_sheet.iter_rows()):
        for col_index, source_cell in enumerate(row):
            target_cell = target_sheet.cell(row=row_index + 1, column=col_index + 1)
            target_cell.fill = copy(source_cell.fill)

    # Save the target workbook
    target_wb.save(target_path)


# Example usage
source_path = "Table-Metrics/Table1.xlsx"
target_path = "test_copy.xlsx"
source_sheet_name = "Table1"
target_sheet_name = "Sheet1"

copy_sheet_colors(source_path, target_path, source_sheet_name, target_sheet_name)

In [7]:
import openpyxl
from openpyxl.styles import Font, Border, PatternFill, Alignment, Protection, NamedStyle


def combine_excel_files(input_files, output_file):
    """
    Combines multiple Excel files into a single Excel file with separate tabs,
    preserving highlighted cells.

    Args:
        input_files (list): A list of paths to the input Excel files.
        output_file (str): The path to the output Excel file.
    """

    new_workbook = openpyxl.Workbook()
    del new_workbook["Sheet"]  # Remove the default sheet

    for file_path in input_files:
        try:
            workbook = openpyxl.load_workbook(file_path)
            sheet = workbook.active
            new_sheet = new_workbook.create_sheet(title=workbook.sheetnames[0])  # use first sheet name as tab title

            for row in sheet.iter_rows():
                for cell in row:
                    new_cell = new_sheet.cell(row=cell.row, column=cell.column, value=cell.value)
                    if cell.has_style:
                        new_cell.font = Font(name=cell.font.name, size=cell.font.size, bold=cell.font.bold, italic=cell.font.italic, color=cell.font.color)
                        new_cell.border = Border(
                            left=cell.border.left,
                            right=cell.border.right,
                            top=cell.border.top,
                            bottom=cell.border.bottom,
                            diagonal=cell.border.diagonal,
                            diagonal_direction=cell.border.diagonal_direction,
                            outline=cell.border.outline,
                            vertical=cell.border.vertical,
                            horizontal=cell.border.horizontal,
                        )
                        new_cell.fill = PatternFill(fill_type=cell.fill.fill_type, start_color=cell.fill.start_color, end_color=cell.fill.end_color)
                        new_cell.number_format = cell.number_format
                        new_cell.protection = Protection(locked=cell.protection.locked, hidden=cell.protection.hidden)
                        new_cell.alignment = Alignment(
                            horizontal=cell.alignment.horizontal,
                            vertical=cell.alignment.vertical,
                            text_rotation=cell.alignment.text_rotation,
                            wrap_text=cell.alignment.wrap_text,
                            shrink_to_fit=cell.alignment.shrink_to_fit,
                            indent=cell.alignment.indent,
                        )

        except FileNotFoundError:
            print(f"Error: File '{file_path}' not found.")
        except Exception as e:
            print(f"An error occurred while processing '{file_path}': {e}")

    new_workbook.save(output_file)
    print(f"Combined Excel file saved to '{output_file}'.")


# Example usage:
# input_files = ["Table-Metrics/tableS64.xlsx"]
# output_file = "combined_files.xlsx"

# try:
#     import openpyxl
# except ImportError:
#     import subprocess

#     subprocess.run(["pip", "install", "openpyxl"])
#     import openpyxl

# combine_excel_files(input_files, output_file)

In [8]:
for file in files[:]:
    parts = str(file).split("/")
    # print(parts)
    if len(parts) >= 2:
        prefix, fig_num = parts[:2]
        fig_name = prefix + "-" + fig_num
        print(fig_name)

Main-Metrics-fig2
Main-Metrics-fig2
Main-Metrics-fig2
Main-Metrics-fig2
Main-Metrics-fig2
Main-Metrics-fig2
Main-Metrics-fig2
Main-Metrics-fig2
Main-Metrics-fig2
Main-Metrics-fig3
Main-Metrics-fig3
Main-Metrics-fig3
Main-Metrics-fig3
Main-Metrics-fig3
Main-Metrics-fig3
Main-Metrics-fig3
Main-Metrics-fig4
Main-Metrics-fig4
Main-Metrics-fig4
Main-Metrics-fig4
Main-Metrics-fig4
Main-Metrics-fig4
Main-Metrics-fig4
Main-Metrics-fig4
Main-Metrics-fig4
Main-Metrics-fig5
Main-Metrics-fig5
Main-Metrics-fig5
Main-Metrics-fig5
Main-Metrics-fig5
Main-Metrics-fig5
Main-Metrics-fig5
Main-Metrics-fig6
Main-Metrics-fig6
Main-Metrics-fig6
Main-Metrics-fig6
Main-Metrics-fig7
Main-Metrics-fig7
Main-Metrics-fig7
Main-Metrics-fig7
Main-Metrics-fig7
Main-Metrics-fig7
Main-Metrics-fig7
Main-Metrics-fig7
Main-Metrics-fig8
Table-Metrics-Table1.xlsx
Table-Sup-Metrics-tableS65.xlsx
Sup-Metrics-S10
Sup-Metrics-S19
Sup-Metrics-S19
Sup-Metrics-S19
Sup-Metrics-S19
Sup-Metrics-S19
Sup-Metrics-S19
Sup-Metrics-S19
Sup-

In [9]:
"Table-Sup" in "Table-Sup-Metrics-tableS64.xlsx"  # .replace("Table-Sup-Metrics-", "")

True

In [10]:
import copy
import openpyxl
from openpyxl import load_workbook

# Create new Excel workbook
workbook = openpyxl.Workbook()
# excel_writer = pd.ExcelWriter("combined_metrics.xlsx", engine="openpyxl")

# Dictionary to store csv_tabs for each figure
fig_data = {}

for file in files[:]:
    parts = str(file).split("/")
    # print(parts)
    if len(parts) >= 2:
        prefix, fig_num = parts[:2]
        fig_name = prefix + "-" + fig_num
        print(parts, fig_name)
        if fig_name.startswith("Main-Metrics"):
            fig_name = fig_name.replace("Main-Metrics-", "").replace("fig", "Fig")
        elif fig_name.startswith("Sup-Metrics"):
            fig_name = fig_name.replace("Sup-Metrics-", "").replace("S", "figS")
        elif "Table-Metrics" in fig_name:
            fig_name = fig_name.replace("Table-Metrics-", "")
            fig_data[fig_name] = {"xlsx": file}
            continue
        elif "Table-Sup" in fig_name:
            fig_name = fig_name.replace("Table-Sup-Metrics-", "")
            fig_data[fig_name] = {"xlsx": file}
            continue
        print(parts, fig_name)
        # Read CSV file
        # Try to peek at CSV columns first
        with open(file) as f:
            header = next(csv.reader(f))
            if "sequence_alignment_heavy" in header:
                print(
                    "Skipping",
                    file,
                    "because it has sequence alignment columns",
                )
                continue

        # If no sequence alignment column, read the CSV
        df = pd.read_csv(file)

        key_list = ["trial", "pseudogroup", "weeks", "pubID"]
        kl = [k for k in key_list if k in df.columns]
        df = df.sort_values(by=kl)

        # Convert both dataframes to csv strings
        csv_tab = [[prefix], [fig_num], [file.name]]
        csv_tab += dataframes_to_csv_strings([df])

        # Add to figure data dictionary
        if fig_name in fig_data:
            fig_data[fig_name].extend(csv_tab)
        else:
            fig_data[fig_name] = csv_tab


# Create tables for each figure
# Sort the fig_data dictionary by figure numbers
# sorted_fig_data = dict(sorted(fig_data.items()))
sorted_fig_data = fig_data

for fig_name, csv_data in sorted_fig_data.items():
    # print(fig_name)

    # Get worksheet or create new one
    if fig_name in workbook.sheetnames:
        sheet = workbook[fig_name]
    else:
        sheet = workbook.create_sheet(fig_name)

    if isinstance(fig_data[fig_name], dict) and "xlsx" in fig_data[fig_name]:
        print("found xlsx")
        xlsx_path = fig_data[fig_name]["xlsx"]
        xlsx_wb = load_workbook(xlsx_path)
        xlsx_sheet = xlsx_wb.active
        for row_idx, row in enumerate(xlsx_sheet.iter_rows(values_only=True), start=len(csv_data) + 1):
            for col_idx, value in enumerate(row, start=1):
                cell = sheet.cell(row=row_idx, column=col_idx, value=value)
                source_cell = xlsx_sheet.cell(row=row_idx - len(csv_data), column=col_idx)
                if source_cell.has_style:
                    if source_cell.font:
                        cell.font = openpyxl.styles.Font(**source_cell.font.__dict__)
                    if source_cell.border:
                        cell.border = openpyxl.styles.Border(**source_cell.border.__dict__)
                    if source_cell.fill:
                        cell.fill = openpyxl.styles.PatternFill(**source_cell.fill.__dict__)
                    if source_cell.number_format:
                        cell.number_format = source_cell.number_format
                    if source_cell.alignment:
                        cell.alignment = openpyxl.styles.Alignment(**source_cell.alignment.__dict__)

        print("added xlsx data")
        continue

    # Write the CSV data to the worksheet
    for row_idx, row_data in enumerate(csv_data, 1):
        if isinstance(row_data, list):
            for col_idx, value in enumerate(row_data, 1):
                sheet.cell(row=row_idx, column=col_idx, value=value)
                # If the fig_data value is a dictionary with 'xlsx' as the key, add a table with the xlsx data

if "Sheet" in workbook.sheetnames:
    workbook.remove(workbook["Sheet"])
workbook.save("combined_metrics.xlsx")

['Main-Metrics', 'fig2', 'fig2_median_all.csv'] Main-Metrics-fig2
['Main-Metrics', 'fig2', 'fig2_median_all.csv'] Fig2
['Main-Metrics', 'fig2', 'figA_percent_gt8$^{++}$_amongigg$^+$_b_cells.csv'] Main-Metrics-fig2
['Main-Metrics', 'fig2', 'figA_percent_gt8$^{++}$_amongigg$^+$_b_cells.csv'] Fig2
['Main-Metrics', 'fig2', 'figB_percent_cd4bs-specific_amongigg$^{+}$_b_cells.csv'] Main-Metrics-fig2
['Main-Metrics', 'fig2', 'figB_percent_cd4bs-specific_amongigg$^{+}$_b_cells.csv'] Fig2
['Main-Metrics', 'fig2', 'figC_percent_of_gt8$^{++}$igg$^{+}$b_cells_that_are_ko$^{-}$.csv'] Main-Metrics-fig2
['Main-Metrics', 'fig2', 'figC_percent_of_gt8$^{++}$igg$^{+}$b_cells_that_are_ko$^{-}$.csv'] Fig2
['Main-Metrics', 'fig2', 'figD_number_of_vrc01-classigg$^{+}$_b_cells.csv'] Main-Metrics-fig2
['Main-Metrics', 'fig2', 'figD_number_of_vrc01-classigg$^{+}$_b_cells.csv'] Fig2
['Main-Metrics', 'fig2', 'figE_percent_of_igg$^{+}$_memory_b_cellsdetected_as_vrc01-class.csv'] Main-Metrics-fig2
['Main-Metrics', 

In [11]:
g001_ptid_pubid = {r.PTID.replace("-", ""): r.PubID for r in pd.read_csv(data.paths.g001_flow_and_seq).itertuples() if r.PubID and r.PTID}
g002_ptid_pubid = {r.ptid.replace("-", ""): r.pubID for r in pd.read_feather(data.paths.g002_flow_and_seq).itertuples() if r.pubID and r.ptid}
g003_ptid_pubid = {r.ptid.replace("-", ""): r.pubID for r in pd.read_feather(data.paths.g003_flow_and_seq).itertuples() if r.pubID and r.ptid}


def update_for_ptidVariants(d):
    for ptid, pubid in [(k, v) for k, v in d.items()]:
        # one dash
        d[ptid[:4] + "-" + ptid[4:]] = pubid
        # complete 2 dashes
        d[ptid[:4] + "-" + ptid[4:6] + "-" + ptid[6:]] = pubid


update_for_ptidVariants(g001_ptid_pubid)
update_for_ptidVariants(g002_ptid_pubid)
update_for_ptidVariants(g003_ptid_pubid)

g00x_ptid_pubid = {**g001_ptid_pubid, **g002_ptid_pubid, **g003_ptid_pubid}

In [12]:
import pandas as pd
from openpyxl import load_workbook
import re


def process_sheet(sheet):
    """Process a single worksheet to replace values"""
    changes = 0
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value and isinstance(cell.value, str):
                for old_val, new_val in g00x_ptid_pubid.items():
                    if old_val in cell.value:
                        cell.value = cell.value.replace(old_val, new_val)
                        changes += 1
    return changes


# Load the workbook
wb = load_workbook("combined_metrics.xlsx")

# Process each sheet
total_changes = 0
for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    changes = process_sheet(sheet)
    total_changes += changes
    print(f"Processed {sheet_name}: {changes} replacements")

# Save the modified workbook
wb.save("combined_metrics_pubID.xlsx")
print(f"Total replacements: {total_changes}")

Processed Fig2: 0 replacements
Processed Fig3: 0 replacements
Processed Fig4: 0 replacements
Processed Fig5: 0 replacements
Processed Fig6: 0 replacements
Processed Fig7: 0 replacements
Processed Fig8: 0 replacements
Processed Table1.xlsx: 0 replacements
Processed tableS65.xlsx: 0 replacements
Processed figS10: 0 replacements
Processed figS19: 0 replacements
Processed figS20: 0 replacements
Processed figS21: 0 replacements
Processed figS22: 0 replacements
Processed figS23: 0 replacements
Processed figS24: 0 replacements
Processed figS25: 0 replacements
Processed figS26: 0 replacements
Processed figS27: 0 replacements
Processed figS28: 297 replacements
Processed figS29: 0 replacements
Processed figS30: 0 replacements
Processed figS31: 0 replacements
Processed figS32: 0 replacements
Processed figS33: 264 replacements
Processed figS34: 0 replacements
Processed figS35: 0 replacements
Processed figS36: 0 replacements
Processed figS39: 0 replacements
Processed figS40: 0 replacements
Processe